In [1]:
from base import util, params
import pandas as pd
import numpy as np
import glob, json
import matplotlib.pyplot as plt
import seaborn as sns
from src import iids_util

### Classifier Model Selection with Low Memory

In [17]:
with open('output/Classifier_Eval_Table_Low_Memory2.json', 'r') as f:
    eval = json.load(f)

table = pd.DataFrame()

for model in eval.keys():
    for metrics in eval.get(model).keys():
        table.loc[model, metrics] = eval.get(model).get(metrics)

for _ in table.columns.tolist():
    table[_] = np.round(table[_].tolist(), 2)

table = table.reset_index(names='models')
table = table.sort_values('matthews_corrcoef', ascending=False).reset_index(drop=True)

table

,models,accuracy,kappa,f1_score,matthews_corrcoef,precision_score,recall_score,runtime
0,AdaptiveRandomForestClassifier,99.02,0.97,97.87,0.97,96.91,98.86,18.5
1,PassiveAggressiveClassifier,98.87,0.97,97.53,0.97,97.61,97.46,13.7
2,StreamingRandomPatches,99.03,0.97,97.90,0.97,97.02,98.80,31.4
3,SGDClassifier,98.66,0.96,97.06,0.96,97.29,96.84,14.1
4,DynamicWeightedMajority,97.98,0.94,95.60,0.94,95.44,95.76,1.2
5,LeveragingBagging,97.97,0.94,95.64,0.94,93.99,97.34,12.4
6,StreamingGradientBoostedTrees,97.79,0.94,95.19,0.94,94.85,95.52,44.7
7,OnlineAdwinBagging,97.47,0.93,94.47,0.93,94.49,94.45,7.4
8,OzaBoost,97.32,0.92,94.10,0.92,94.60,93.62,1.7
9,OnlineBagging,96.29,0.90,91.97,0.90,91.11,92.84,5.7


In [18]:
from capymoa.stream import NumpyStream

In [33]:
feature, target, header = iids_util.loading_edge_dataset(base_device='modbus', 
                             all_in_fusion=True,
                             load_all=False,
                             low_memory=True,
                             sample_size=.1,
                             random_seed=80)

print("feature dtype: ",feature.dtype)

scaled = iids_util.online_normalization(data=feature,
                                        window_size=60,
                                        scaler_model='StandardScaler')

target = iids_util.map_as_binary_class(ntarget=target,
                                       class_0='normal')

stream_cls = NumpyStream(X=scaled, y=target)

feature dtype:  float16


In [34]:
curr = stream_cls.next_instance()

In [37]:
curr.x.dtype

dtype('float64')

### tab3.3 Variabel Names

In [30]:
# tab3.3
table = pd.DataFrame()
for dev in params.feature_naming.keys():
    for variable in params.feature_naming.get(dev):
        table.loc[len(table)+1, ['Perangkat Edge', 'Feature', 'Deskripsi']] = [dev.replace('_',' ').title(), variable.upper(), 'xx']
table = table.reset_index(names='No')
table = table.sort_values('Perangkat Edge')
util.to_latex_table(data=table, fname='output/tab3.3.tex')

### Correlation Heatmaps

In [ ]:
# Corr Heatmap Fridge
layer = 'Edge'
for dev in params.edge_device_list:
    df = pd.read_csv(params.edge_fusion_csv_sources.get(dev), low_memory=False)
    df = df.drop(columns=['ts', 'target']).copy()
    dst = f"output/corr_{layer}_{dev}.png"
    corr = df.corr()
    plt.figure(figsize=(10, 6)) # Optional: Adjusts the size of the plot
    graf = sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=.5, fmt='.3f', annot_kws={"size": 7})
    graf.set_xticklabels(graf.get_xticklabels(), rotation=30, fontsize=8)
    graf.set_yticklabels(graf.get_yticklabels(), fontsize=8)
    dev = dev.replace('_',' ').title()
    plt.title(f'Heatmap of Correlations - {layer.title()} {dev} Dataset')
    # plt.show()
    plt.savefig(dst)

### Drift Detector

In [ ]:
from capymoa.drift.detectors import ADWIN
np.random.seed(0)

devlist = params.edge_device_list
cls = params.normal_label
table = pd.DataFrame()
for dev in devlist:
    varname = params.feature_naming.get(dev) + [params.target_label]
    df = pd.read_csv(params.edge_fusion_csv_sources.get(dev), low_memory=False)
    df = df[varname].copy()
    cls = list(df[params.target_label].unique())
    for fitur in params.feature_naming.get(dev):
        for state in cls:
            data = df[df[params.target_label] == state][fitur].tolist()
            drift_num = 0
            detector = ADWIN()
            for val in data:
                detector.add_element(val)
                if detector.detected_change():
                    drift_num += 1
            table.loc[fitur, state] = drift_num
table = table.fillna('-').reset_index(names='Feature')
util.to_latex_table(data=table, fname='output/tab.tex')

### Labeling in Reports

In [16]:
dev_id = {'fridge': 'E1',
          'garage_door': 'E2',
          'modbus': 'E3',
          'motion_light': 'E4',
          'thermostat': 'E5',
          'weather': 'E6'}

In [23]:
table = pd.DataFrame(columns=['Nama', 'Kelompok', 'Kode'])
for dev in dev_id.keys():
    fitur = params.feature_naming.get(dev)
    num = 1
    prefix = dev_id.get(dev)
    for _ in fitur:
        name = dev.replace('_', ' ').title()
        fitur = _.replace('_', ' ').title()
        table.loc[len(table), ['Nama', 'Kelompok', 'Kode']] = [fitur, name, f'{prefix}.{num}']
        num += 1

In [24]:
num = 1
for _ in ['Autoencoder', 'HalfSpaceTrees', 'AdaptiveIsolationForest']:
    table.loc[len(table), ['Nama', 'Kelompok', 'Kode']] = [_, 'Anomaly detector', f'D_{num}']
    num += 1

In [25]:
num = 1
for _ in ['DynamicWeightedMajority', 'OzaBoost', 'HoeffdingAdaptiveTree']:
    table.loc[len(table), ['Nama', 'Kelompok', 'Kode']] = [_, 'Classifier', f'C_{num}']
    num += 1

In [38]:
src = glob.glob('data_src/ToN_IoT/SecuityEvents_GroundTruth_datasets/SecurityEvents_IoT_datasets/*.csv')
cls = []
for i in src:
    df = pd.read_csv(i)
    cls.extend(df['type'].unique().tolist())
del df
cls = list(set(cls))
cls.sort()
cls = ['normal'] + cls
num = 0
for _ in cls:
    table.loc[len(table), ['Nama', 'Kelompok', 'Kode']] = [_, 'Kelas Ancaman', f'cls_{num}']
    num += 1

In [39]:
table

,Nama,Kelompok,Kode
0,Fridge Temperature,Fridge,E1.1
1,Temp Condition,Fridge,E1.2
2,Door State,Garage Door,E2.1
3,Sphone Signal,Garage Door,E2.2
4,Fc1 Read Input Register,Modbus,E3.1
5,Fc2 Read Discrete Value,Modbus,E3.2
6,Fc3 Read Holding Register,Modbus,E3.3
7,Fc4 Read Coil,Modbus,E3.4
8,Motion Status,Motion Light,E4.1
9,Light Status,Motion Light,E4.2


In [40]:
table = table[['Kelompok', 'Nama', 'Kode']].copy()

In [42]:
util.to_latex_table(table, 'output/tab.tex')